### Importing libraries

In [1]:
import math

pixels_width = 1920
pixels_height = 1080
screen_width_inches = 21
screen_height_inches = 11

ppi = math.sqrt(pixels_width**2 + pixels_height**2) / math.sqrt(screen_width_inches**2 + screen_height_inches**2)

print(f"Pixel Per Inch (PPI): {ppi}")

Pixel Per Inch (PPI): 92.92403050255547


In [2]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np
from decimal import Decimal
import scipy.ndimage
import matplotlib.pyplot as plt

In [3]:
# file_path1 = 'plusLenses/amblypiaStudySanaLeft4.xlsx'
# # file_path2 = 'TestData/amblyopiaStudy MetricsSanaTest.tsv'

# dataExport = pd.read_excel(file_path1)
# dataMetric = pd.read_csv(file_path2, sep='\t')

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 10)


In [ ]:

# dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)

In [ ]:
# dfSaccade[["Eyetracker timestamp [μs]",  "Presented Stimulus name",  "Eye movement type", "Gaze point X [MCS norm]", "Gaze point Y [MCS norm]", 
#         'Gaze point left X [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right X [MCS norm]', 
#                          'Gaze point right Y [MCS norm]']].head()

In [ ]:
# display(dataExport.columns[0:50])
# print(dataExport['Recording resolution height'])
# print(dataExport['Recording resolution width'])
# print(dataExport['Viewport position X'])
# print(dataExport['Viewport width'])
# print(dataExport['Original Media height'])

Checking to find the end of calibration index

In [12]:
from sklearn.impute import KNNImputer

def preprocessing(dataExport):
    imputer = KNNImputer(n_neighbors=2)

    #dataExport[dataExport.eq("Eyetracker Calibration").any(1)]['Presented Stimulus name']
    calibrationEnd = dataExport[dataExport.eq("Eyetracker Calibration").any(1)]['Presented Stimulus name'].index.to_list()[-1]
    calibrationEnd+=1
    
    dataExport = dataExport[["Eyetracker timestamp [μs]", "Event", "Event value", "Gaze point X [MCS norm]", "Gaze point Y [MCS norm]", "Eye movement type", 
    "Presented Stimulus name", 'AOI hit [slothStimulus-final (1) - sloth1]',
       'AOI hit [slothStimulus-final (1) - sloth2]',
       'AOI hit [slothStimulus-final (1) - sloth3]',
       'AOI hit [slothStimulus-final (1) - sloth4]',
       'AOI hit [slothStimulus-final (1) - sloth5]',
       'AOI hit [slothStimulus-final (1) - sloth6]',
       'AOI hit [slothStimulus-final (1) - sloth7]',
       'AOI hit [slothStimulus-final (1) - sloth8]',
       'AOI hit [slothStimulus-final (1) - sloth9]',
       'AOI hit [slothStimulus-final (1) - sloth10]',
       'AOI hit [slothStimulus-final (1) - sloth11]',
       'AOI hit [catStimulus-final (1) - cat1]',
        'AOI hit [catStimulus-final (1) - cat2]',
       'AOI hit [catStimulus-final (1) - cat3]',
       'AOI hit [catStimulus-final (1) - cat4]',
       'AOI hit [catStimulus-final (1) - cat5]',
       'AOI hit [catStimulus-final (1) - cat6]',
       'AOI hit [catStimulus-final (1) - cat7]',
       'AOI hit [catStimulus-final (1) - cat8]',
       'AOI hit [catStimulus-final (1) - cat9]',
       'AOI hit [catStimulus-final (1) - cat10]',
       'AOI hit [catStimulus-final (1) - cat11]',
        'Gaze point left X [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right X [MCS norm]', 
                         'Gaze point right Y [MCS norm]', 
        'Gaze point left X [DACS px]', 'Gaze point left Y [DACS px]', 'Gaze point right X [DACS px]', 
                         'Gaze point right Y [DACS px]',  'Gaze point X [DACS px]',  'Gaze point Y [DACS px]'
                            ]]



    #Only isolate non calibration data
    dataExportWithoutCalibration = dataExport[calibrationEnd:]

    #Drop all columns with eye movement type of eyes not found
    dataExportWithoutCalibration = dataExportWithoutCalibration[dataExportWithoutCalibration['Eye movement type'] != 'EyesNotFound']

    #Convert the x and y coordinates into pixels and then into inches using ppi of 92.
    
    #Check this -
    #Normalized --> absolute pixels on the screen - ()
    dataExportWithoutCalibration[[ 'Gaze point X [MCS norm]', 'Gaze point left X [MCS norm]', 'Gaze point right X [MCS norm]']] = dataExportWithoutCalibration[[ 'Gaze point X [MCS norm]', 'Gaze point left X [MCS norm]', 'Gaze point right X [MCS norm]']]*1920/92 

    dataExportWithoutCalibration[[ 'Gaze point Y [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right Y [MCS norm]']] = dataExportWithoutCalibration[[ 'Gaze point Y [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right Y [MCS norm]']]*1080/92

    #Find the first timestamp that isn't nan: 
    # nonNanTimeStampSlothStart = dataExportWithoutCalibration['Eyetracker timestamp [μs]'].iloc[0]
    nonNanTimeStampSlothStart = dataExportWithoutCalibration.loc[dataExportWithoutCalibration['Eyetracker timestamp [μs]'].first_valid_index(), 'Eyetracker timestamp [μs]']

    #normalizes TIME when the calibration ends and stimulus sloth starts (at 0)
    dataExportWithoutCalibration['Eyetracker timestamp [μs]'] = dataExportWithoutCalibration['Eyetracker timestamp [μs]'] - nonNanTimeStampSlothStart #dataExportWithoutCalibration['Eyetracker timestamp [μs]'].iloc[0]

    #normalizes TIME when the sloth ends and cats start (at 0)
    #find the index where the cat stimulus begins
    indices = dataExportWithoutCalibration.index[dataExportWithoutCalibration['Presented Stimulus name'] == "catStimulus-final (1)"]

    #Find the beginning time for normalization
    beginTime = dataExportWithoutCalibration['Eyetracker timestamp [μs]'].loc[indices[0]]
    index_of_catBegin = (dataExportWithoutCalibration.index.tolist()).index(indices[0])
    onylCat = dataExportWithoutCalibration.index[index_of_catBegin:]
    #iterate through all times for cat stimulus and update
    for i in onylCat:
        dataExportWithoutCalibration.loc[i, 'Eyetracker timestamp [μs]'] = dataExportWithoutCalibration.loc[i, 'Eyetracker timestamp [μs]'] - beginTime

    

    #interpolate data
    columns_to_interpolate = ['Gaze point X [MCS norm]', 'Gaze point Y [MCS norm]', 'Gaze point left X [MCS norm]', 
                          'Gaze point left Y [MCS norm]', 'Gaze point right X [MCS norm]', 'Gaze point right Y [MCS norm]', 
                              'Gaze point left X [DACS px]', 'Gaze point left Y [DACS px]', 'Gaze point right X [DACS px]', 
                         'Gaze point right Y [DACS px]',  'Gaze point X [DACS px]',  'Gaze point Y [DACS px]'
                             ]
    
    

    # Use the subset parameter to interpolate only the specified columns

    dataExportWithoutCalibration[columns_to_interpolate] = dataExportWithoutCalibration[columns_to_interpolate].interpolate(method='polynomial', order =2 )
#     dataExportWithoutCalibration['Gaze point right X [MCS norm]'] = imputer.fit_transform(dataExportWithoutCalibration[['Gaze point right X [MCS norm]']])
#     dataExportWithoutCalibration['Gaze point right Y [MCS norm]'] = imputer.fit_transform(dataExportWithoutCalibration[['Gaze point right X [MCS norm]']])


    nan_df = dataExportWithoutCalibration.isna().sum()  # or df.isnull()
    print("LOOK HERE")
    print(nan_df)
    
#     display(dataExportWithoutCalibration['Gaze point right X [MCS norm]'][0:100])
    #isolate only saccade data
    mask = dataExportWithoutCalibration["Eye movement type"] == "Saccade"
    dfSaccade = dataExportWithoutCalibration[mask]
    excel_file_name = "thing"
#     dfSaccade.to_excel(os.path.join(os.getcwd(), excel_file_name), index=False)
    dfSaccade.to_excel(os.path.join(os.getcwd(), excel_file_name + ".xlsx"), index=False, engine='xlsxwriter')

    
#     display(dfSaccade.head())
#     display(dfSaccade.tail())
    return dfSaccade, dataExportWithoutCalibration



In [13]:
def primaryAndSecondarySaccade(dfSaccade, times_dict): 
    AOI_list = ['AOI hit [slothStimulus-final (1) - sloth1]',
           'AOI hit [slothStimulus-final (1) - sloth2]',
           'AOI hit [slothStimulus-final (1) - sloth3]',
           'AOI hit [slothStimulus-final (1) - sloth4]',
           'AOI hit [slothStimulus-final (1) - sloth5]',
           'AOI hit [slothStimulus-final (1) - sloth6]',
           'AOI hit [slothStimulus-final (1) - sloth7]',
           'AOI hit [slothStimulus-final (1) - sloth8]',
           'AOI hit [slothStimulus-final (1) - sloth9]',
           'AOI hit [slothStimulus-final (1) - sloth10]',
           'AOI hit [slothStimulus-final (1) - sloth11]',
           'AOI hit [catStimulus-final (1) - cat1]',
            'AOI hit [catStimulus-final (1) - cat2]',
           'AOI hit [catStimulus-final (1) - cat3]',
           'AOI hit [catStimulus-final (1) - cat4]',
           'AOI hit [catStimulus-final (1) - cat5]',
           'AOI hit [catStimulus-final (1) - cat6]',
           'AOI hit [catStimulus-final (1) - cat7]',
           'AOI hit [catStimulus-final (1) - cat8]',
           'AOI hit [catStimulus-final (1) - cat9]',
           'AOI hit [catStimulus-final (1) - cat10]',
          'AOI hit [catStimulus-final (1) - cat11]']

    aoi_cat =  ['AOI hit [catStimulus-final (1) - cat1]',
            'AOI hit [catStimulus-final (1) - cat2]',
           'AOI hit [catStimulus-final (1) - cat3]',
           'AOI hit [catStimulus-final (1) - cat4]',
           'AOI hit [catStimulus-final (1) - cat5]',
           'AOI hit [catStimulus-final (1) - cat6]',
           'AOI hit [catStimulus-final (1) - cat7]',
           'AOI hit [catStimulus-final (1) - cat8]',
           'AOI hit [catStimulus-final (1) - cat9]',
           'AOI hit [catStimulus-final (1) - cat10]',
          'AOI hit [catStimulus-final (1) - cat11]']




    primarySaccadeIndex = defaultdict(int)
    primarySaccadeTime = defaultdict(int)
    dfSaccadeReset = dfSaccade.reset_index()
    index_of_catBegin = dfSaccadeReset[dfSaccadeReset['Presented Stimulus name'] == "catStimulus-final (1)"].iloc[0, 0]
    

    for i in times_dict.keys():
        for index, row in dfSaccade.iterrows():
            time = row['Eyetracker timestamp [μs]']
            if i in aoi_cat:
                if index > index_of_catBegin: 
                    if time/1000000 > (times_dict[i]) and (time/1000000 < (times_dict[i] + 2.5)): 
                        primarySaccadeIndex[i] = index
                        primarySaccadeTime[i] = time/1000000
                        break


            else: 
                if time/1000000 > (times_dict[i]) and (time/1000000 < (times_dict[i] + 2.5)): 
                    primarySaccadeIndex[i] = index
                    primarySaccadeTime[i] = time/1000000
                    break
    primarySaccadeTime
   


    ## TIME + AOI SECONDARY SACCADES
    secondarySaccadesTimeAndAOI = defaultdict(int)
    for i in times_dict.keys():
        all_possible_hits = dfSaccade[dfSaccade[i] == np.max(dfSaccade[i])]
        for index, row in all_possible_hits.iterrows(): 
            if i in aoi_cat:
                if index > index_of_catBegin:
                    time = row['Eyetracker timestamp [μs]']
                    if (time/1000000 > (primarySaccadeTime[i]+0.2)) and (time/1000000 < (times_dict[i] + 2.5)):
                        secondarySaccadesTimeAndAOI[i] = index
                        break
            else:
                time = row['Eyetracker timestamp [μs]']
                if (time/1000000 > (primarySaccadeTime[i]+0.2)) and (time/1000000 < (times_dict[i] + 2.5)):
                    secondarySaccadesTimeAndAOI[i] = index
                    break
#     display(secondarySaccadesTimeAndAOI)
#     display(primarySaccadeIndex)


    secondarySaccadesTime = defaultdict(int)
    for i in times_dict.keys():
        for index, row in dfSaccade.iterrows():
            if i in aoi_cat:
                if index > index_of_catBegin:
                    time = row['Eyetracker timestamp [μs]']
                    if (time/1000000 > (0.200 + primarySaccadeTime[i])) and (time/1000000 < (times_dict[i] + 2.5)):
                        secondarySaccadesTime[i] = index
                        break
            else:
                time = row['Eyetracker timestamp [μs]']
                if (time/1000000 > (0.200 + primarySaccadeTime[i])) and (time/1000000 < (times_dict[i] + 2.5)):
                    secondarySaccadesTime[i] = index
                    break

    return primarySaccadeTime, primarySaccadeIndex, secondarySaccadesTime, secondarySaccadesTimeAndAOI


In [14]:
# dataExport[calibrationEnd:]
# dataExport['Eyetracker timestamp [μs]'].isna().sum()
# dataExport['Eyetracker timestamp [μs]'].count()
# dfSaccade['Eyetracker timestamp [μs]'].isna().sum()

### Identify the primary saccade 

In [15]:
# ppi = math.sqrt(1080**2 + 1920**2) / 24
# print(ppi)
# viewing_dist = 22*ppi
# print(viewing_dist)

#convert the normalizaed values into pixels and then into 
#we have l is 22 inches


In [16]:
def find_closest_values(lst, target):
    closest_lower = None
    closest_higher = None

    for value in lst:
        if value <= target:
            if closest_lower is None or target - value < target - closest_lower:
                closest_lower = value
        if value >= target:
            if closest_higher is None or value - target < closest_higher - target:
                closest_higher = value

    return closest_lower, closest_higher


#### Finding amplitude for both eyes, right, eye, left eye, for primary saccades. 

In [38]:
def amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration): 
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in primarySaccadeIndex.items():
        fixation_list = dataExportWithoutCalibration
        mask = dataExportWithoutCalibration["Eye movement type"] == "Fixation"
        dfFixation = dataExportWithoutCalibration[mask]
        
        fix1, fix2 = find_closest_values(dfFixation.index.tolist(), value) 
        if fix1 is None: 
            fix1 = value


        if np.isnan(value)==False:
            row1 = df_reset_orginal[df_reset_orginal['index'] == fix1]
#             display(df_reset_orginal)
            x1 = Decimal(str(row1['Gaze point X [DACS px]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [DACS px]'].iloc[0]))

            #convert the index label to actual index
            
#             index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == fixation1].index
#             print("start")
#             print(index_of_desired_row)
#             index_of_desired_row = index_of_desired_row -1
#             new_value = df_reset_orginal.loc[index_of_desired_row, 'index']
#             print(new_value)
            row2 = df_reset_orginal[df_reset_orginal['index'] == int(fix2)]
            x2 = Decimal(str(row2.loc[:,'Gaze point X [DACS px]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [DACS px]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (180/np.pi) * np.arctan(distance/(28*92))

            x1 = row1['Gaze point left X [DACS px]'].iloc[0]
            y1 = row1['Gaze point left Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            
            amplitudeLeftEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))
            
            
            
            x1 = row1['Gaze point right X [DACS px]'].iloc[0]
            y1 = row1['Gaze point right Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [DACS px]'].iloc[0]
            print(x1,y1,x2,y2)
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))


   
    
    print("Primary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    values = [x for x in values if not np.isnan(x)]
    print(np.mean(values), np.std(values))
    print(" ")
    meanBothEyes, stdBothEyes = np.mean(values), np.std(values)**2
    print("Primary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    values = [x for x in values if not np.isnan(x)]
    print(np.mean(values), np.std(values))
    print(" ")
    meanLeftEyes, stdLeftEyes = np.mean(values), np.std(values)**2

    print("Primary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    values = [x for x in values if not np.isnan(x)]
    print(np.mean(values), np.std(values))
    print(" ")
    meanRightEyes, stdRightEyes = np.mean(values), np.std(values)**2

    
    display(amplitudeBothEyesDict)
    print("RIGHT")
    display(amplitudeRightEyeDict)
    print(" ")
    print("LEFT")
    display(amplitudeLeftEyeDict)


    return (meanBothEyes, stdBothEyes), (meanLeftEyes, stdLeftEyes), (meanRightEyes, stdRightEyes)

### Finding amplitude for both eyes, right eye, left eye for secondary saccades 



### Secondary saccades (only time constraint)

In [39]:
def secondarySaccadeTimeAmplitude(dfSaccade, dataExportWithoutCalibration, secondarySaccadesTime): 
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in secondarySaccadesTime.items():
        fixation_list = dataExportWithoutCalibration
        mask = dataExportWithoutCalibration["Eye movement type"] == "Fixation"
        dfFixation = dataExportWithoutCalibration[mask]
        fix1, fix2 = find_closest_values(dfFixation.index.tolist(), value) 
        if fix1 is None: 
            fix1 = value
        if np.isnan(value)==False:
            
            row1 = df_reset_orginal[df_reset_orginal['index'] == fix1]
            x1 = Decimal(str(row1['Gaze point X [DACS px]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [DACS px]'].iloc[0]))

            #convert the index label to actual index
            
#             index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == fixation1].index
#             print("start")
#             print(index_of_desired_row)
#             index_of_desired_row = index_of_desired_row -1
#             new_value = df_reset_orginal.loc[index_of_desired_row, 'index']
#             print(new_value)
            row2 = df_reset_orginal[df_reset_orginal['index'] == int(fix2)]
            x2 = Decimal(str(row2.loc[:,'Gaze point X [DACS px]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [DACS px]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (180/np.pi) * np.arctan(distance/(28*92))

            x1 = row1['Gaze point left X [DACS px]'].iloc[0]
            y1 = row1['Gaze point left Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            
            amplitudeLeftEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))
            
            

            x1 = row1['Gaze point right X [DACS px]'].iloc[0]
            y1 = row1['Gaze point right Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))



    print("Secondary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    values = [x for x in values if not np.isnan(x)]

    print(np.mean(values), np.std(values))
    print(" ")
    meanBothEyes, stdBothEyes = np.mean(values), np.std(values)**2
    print("Secondary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    values = [x for x in values if not np.isnan(x)]
    print(np.mean(values), np.std(values))
    print(" ")
    meanLeftEyes, stdLeftEyes = np.mean(values), np.std(values)**2

    print("Secondary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    values = [x for x in values if not np.isnan(x)]
    print(np.mean(values), np.std(values))
    print(" ")
    meanRightEyes, stdRightEyes = np.mean(values), np.std(values)**2

    
    display(amplitudeBothEyesDict)
    print("RIGHT")
    display(amplitudeRightEyeDict)
    print(" ")
    print("LEFT")
    display(amplitudeLeftEyeDict)


    return (meanBothEyes, stdBothEyes), (meanLeftEyes, stdLeftEyes), (meanRightEyes, stdRightEyes)

### Secondary saccades (time and AOI constraint)

In [40]:
def secondarySaccadeTimeandAOIAmplitude(dfSaccade, dataExportWithoutCalibration, secondarySaccadesTimeAndAOI):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in secondarySaccadesTimeAndAOI.items():
        fixation_list = dataExportWithoutCalibration
        mask = dataExportWithoutCalibration["Eye movement type"] == "Fixation"
        dfFixation = dataExportWithoutCalibration[mask]
        fix1, fix2 = find_closest_values(dfFixation.index.tolist(), value) 
        if fix1 is None: 
            fix1 = value

        if np.isnan(value)==False:
            
            row1 = df_reset_orginal[df_reset_orginal['index'] == fix1]
            x1 = Decimal(str(row1['Gaze point X [DACS px]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [DACS px]'].iloc[0]))

            #convert the index label to actual index
            
#             index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == fixation1].index
#             print("start")
#             print(index_of_desired_row)
#             index_of_desired_row = index_of_desired_row -1
#             new_value = df_reset_orginal.loc[index_of_desired_row, 'index']
#             print(new_value)
            row2 = df_reset_orginal[df_reset_orginal['index'] == int(fix2)]
            x2 = Decimal(str(row2.loc[:,'Gaze point X [DACS px]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [DACS px]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (180/np.pi) * np.arctan(distance/(28*92))

            x1 = row1['Gaze point left X [DACS px]'].iloc[0]
            y1 = row1['Gaze point left Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            
            amplitudeLeftEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))
            
            

            x1 = row1['Gaze point right X [DACS px]'].iloc[0]
            y1 = row1['Gaze point right Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))



    print("Secondary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    values = [x for x in values if not np.isnan(x)]
    print(np.mean(values), np.std(values))
    print(" ")
    meanBothEyes, stdBothEyes = np.mean(values), np.std(values)**2
    print("Secondary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    values = [x for x in values if not np.isnan(x)]
    print(np.mean(values), np.std(values))
    print(" ")
    meanLeftEyes, stdLeftEyes = np.mean(values), np.std(values)**2

    print("Secondary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    values = [x for x in values if not np.isnan(x)]
    print(np.mean(values), np.std(values))
    print(" ")
    meanRightEyes, stdRightEyes = np.mean(values), np.std(values)**2

    
    display(amplitudeBothEyesDict)
    print("RIGHT")
    display(amplitudeRightEyeDict)
    print(" ")
    print("LEFT")
    display(amplitudeLeftEyeDict)


    return (meanBothEyes, stdBothEyes), (meanLeftEyes, stdLeftEyes), (meanRightEyes, stdRightEyes)

### Finding Saccade Latency - using given timestamps

In [20]:
def saccadeLatency(primarySaccadeTime, times_dict):
    saccadeLatency =  defaultdict(int)
    i = 0
    for key,value in primarySaccadeTime.items(): 
        saccadeLatency[key] = abs(times_dict[key] - value)
    
    print("Primary Saccade Latency (Mean and STD)")
    values = list(saccadeLatency.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return (np.mean(values), np.std(values)**2)



### Accuracy between primary saccade and stimuli: 

In [21]:
def accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    #}

    #convert pixel values into inches
    for key,value in stimuli_location.items(): 
        stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in primarySaccadeIndex.items():
        if np.isnan(value)==False:
            
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point Y [MCS norm]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            accuracy_dist[key] = distance
    
    print("Distance between primary saccade & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")

    return (np.mean(values), np.std(values)**2)
    

### Accuracy between secondary saccade (time constraint) and stimuli: 

In [22]:
def accuracySecondarySaccadeTime(secondarySaccadesTime, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    # }

    #convert pixel values into inches
    for key,value in stimuli_location.items(): 
        stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in secondarySaccadesTime.items():
        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point Y [MCS norm]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            accuracy_dist[key] = distance


    
    print("Distance between secondary saccade (time only) & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return (np.mean(values), np.std(values)**2)

### Accuracy between secondary saccade (time and AOI constraint) and stimuli: 

In [23]:
def accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    # }

    #convert pixel values into inches
    for key,value in stimuli_location.items(): 
        stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in secondarySaccadesTimeAndAOI.items():
        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point Y [MCS norm]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            accuracy_dist[key] = distance

    print("Distance between secondary saccade (time + AOI only) & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return (np.mean(values), np.std(values)**2)

### Corrective saccades between primary saccade and secondary saccade (time + AOI) 

In [24]:
def correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    correctiveSaccade = defaultdict(int)

    for key,value in secondarySaccadesTimeAndAOI.items():

        index_primary = df_reset[df_reset['index'] == primarySaccadeIndex[key]].index[0]
        index_secondary = df_reset[df_reset['index'] == value].index[0]
        correctiveSaccade[key] = int(index_secondary) - int(index_primary)

    print("Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)")
    values = list(correctiveSaccade.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return (np.mean(values), np.std(values)**2)

### Corrective saccades between primary saccade and secondary saccade (time) 

In [25]:
def correctiveSaccadeTime(secondarySaccadesTime, primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    correctiveSaccade = defaultdict(int)

    for key,value in secondarySaccadesTime.items():

        index_primary = df_reset[df_reset['index'] == primarySaccadeIndex[key]].index
        if len(index_primary) != 0:
            index_secondary = df_reset[df_reset['index'] == value].index[0]

            correctiveSaccade[key] = int(index_secondary) - int(index_primary[0])

    print("Number of corrective saccade between secondary saccade (time only) & primary (Mean and STD)")
    values = list(correctiveSaccade.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return (np.mean(values), np.std(values)**2)

## MAIN LOOP

In [41]:
amplitudeBothEyesDictPrimaryRIGHT = []
amplitudeBothEyesDictSecondaryRIGHT = []
amplitudeRightEyesDictPrimaryRIGHT = []
amplitudeRightEyesDictSecondaryRIGHT = []
amplitudeLeftEyesDictPrimaryRIGHT = []
amplitudeLeftEyesDictSecondaryRIGHT = []
saccade_latencyRIGHT = []
accuracy_distPrimaryRIGHT = []
accuracy_distSecondaryRIGHT = []
correctiveSaccadeRIGHT = []

amplitudeBothEyesDictPrimaryLEFT = []
amplitudeBothEyesDictSecondaryLEFT = []
amplitudeRightEyesDictPrimaryLEFT = []
amplitudeRightEyesDictSecondaryLEFT = []
amplitudeLeftEyesDictPrimaryLEFT= []
amplitudeLeftEyesDictSecondaryLEFT= []
saccade_latencyLEFT = []
accuracy_distPrimaryLEFT = []
accuracy_distSecondaryLEFT = []
correctiveSaccadeLEFT = []

amplitudeBothEyesDictPrimaryNormal = []
amplitudeBothEyesDictSecondaryNormal = []
amplitudeRightEyesDictPrimaryNormal = []
amplitudeRightEyesDictSecondaryNormal = []
amplitudeLeftEyesDictPrimaryNormal= []
amplitudeLeftEyesDictSecondaryNormal= []
saccade_latencyNormal = []
accuracy_distPrimaryNormal = []
accuracy_distSecondaryNormal = []
correctiveSaccadeNormal = []

amplitudeBothEyesDictPrimaryRIGHTVariance = []
amplitudeBothEyesDictSecondaryRIGHTVariance = []
amplitudeRightEyesDictPrimaryRIGHTVariance = []
amplitudeRightEyesDictSecondaryRIGHTVariance = []
amplitudeLeftEyesDictPrimaryRIGHTVariance = []
amplitudeLeftEyesDictSecondaryRIGHTVariance = []
saccade_latencyRIGHTVariance = []
accuracy_distPrimaryRIGHTVariance = []
accuracy_distSecondaryRIGHTVariance = []
correctiveSaccadeRIGHTVariance = []

amplitudeBothEyesDictPrimaryLEFTVariance = []
amplitudeBothEyesDictSecondaryLEFTVariance = []
amplitudeRightEyesDictPrimaryLEFTVariance = []
amplitudeRightEyesDictSecondaryLEFTVariance = []
amplitudeLeftEyesDictPrimaryLEFTVariance= []
amplitudeLeftEyesDictSecondaryLEFTVariance= []
saccade_latencyLEFTVariance = []
accuracy_distPrimaryLEFTVariance = []
accuracy_distSecondaryLEFTVariance = []
correctiveSaccadeLEFTVariance = []

amplitudeBothEyesDictPrimaryNormalVariance = []
amplitudeBothEyesDictSecondaryNormalVariance = []
amplitudeRightEyesDictPrimaryNormalVariance = []
amplitudeRightEyesDictSecondaryNormalVariance = []
amplitudeLeftEyesDictPrimaryNormalVariance= []
amplitudeLeftEyesDictSecondaryNormalVariance= []
saccade_latencyNormalVariance = []
accuracy_distPrimaryNormalVariance = []
accuracy_distSecondaryNormalVariance = []
correctiveSaccadeNormalVariance = []

In [42]:
import os
import glob

times_dict = {'AOI hit [slothStimulus-final (1) - sloth1]' : 0,
           'AOI hit [slothStimulus-final (1) - sloth2]': 2.5,
           'AOI hit [slothStimulus-final (1) - sloth3]': 5,
           'AOI hit [slothStimulus-final (1) - sloth4]': 7.5,
           'AOI hit [slothStimulus-final (1) - sloth5]': 10,
           'AOI hit [slothStimulus-final (1) - sloth6]': 12.5,
           'AOI hit [slothStimulus-final (1) - sloth7]': 15,
           'AOI hit [slothStimulus-final (1) - sloth8]': 17.5,
           'AOI hit [slothStimulus-final (1) - sloth9]': 20,
           'AOI hit [slothStimulus-final (1) - sloth10]': 22.5,
           'AOI hit [slothStimulus-final (1) - sloth11]': 25,
           'AOI hit [catStimulus-final (1) - cat1]': 0,
           'AOI hit [catStimulus-final (1) - cat2]': 2.5,
           'AOI hit [catStimulus-final (1) - cat3]': 5,
           'AOI hit [catStimulus-final (1) - cat4]': 7.5,
           'AOI hit [catStimulus-final (1) - cat5]': 10,
           'AOI hit [catStimulus-final (1) - cat6]': 12.5,
           'AOI hit [catStimulus-final (1) - cat7]': 15,
           'AOI hit [catStimulus-final (1) - cat8]': 17.5,
           'AOI hit [catStimulus-final (1) - cat9]': 20,
           'AOI hit [catStimulus-final (1) - cat10]': 22.5,
           'AOI hit [catStimulus-final (1) - cat11]': 25}

# plusLenseRight175 = defaultdict(int)
# plusLenseLeft175 = defaultdict(int)
# normal = defaultdict(int)
#dataCollection = [normal, plusLenseLeft175, plusLenseRight175]
# dataCollection = defaultdict(int) 



# folder_path = 'plusLenses'
# excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
# print(excel_files)
folder_names = ['11-29-data/Normal', '11-29-data/Right', '11-29-data/Left']

files = glob.glob(os.path.join(folder_names[1], '*.xlsx'))
for idx, file_path in enumerate(files):
    print(file_path)
    dataCollection = defaultdict(int) 
    dataExport = pd.read_excel(file_path)
    dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)
    primarySaccadeTime, primarySaccadeIndex,secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
    bothEyesPrimary, leftEyesPrimary, rightEyesPrimary = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
    bothEyesSecondary, leftEyesSecondary, rightEyesSecondary = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
    saccadeLatencyValues = saccadeLatency(primarySaccadeTime, times_dict)
    #dataCollection['accuracy_distPrimary'] = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
    #dataCollection['accuracy_distSecondary'] = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
    correctiveSaccadeValues = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
    amplitudeBothEyesDictPrimaryRIGHT.append(bothEyesPrimary[0])
    amplitudeBothEyesDictSecondaryRIGHT.append(bothEyesSecondary[0])
    amplitudeRightEyesDictPrimaryRIGHT.append(rightEyesPrimary[0])
    amplitudeRightEyesDictSecondaryRIGHT.append(rightEyesSecondary[0])
    amplitudeLeftEyesDictPrimaryRIGHT.append(leftEyesPrimary[0])
    amplitudeLeftEyesDictSecondaryRIGHT.append(leftEyesSecondary[0])
    saccade_latencyRIGHT.append(saccadeLatencyValues[0])
#     accuracy_distPrimaryRIGHT.append(dataCollection['accuracy_distPrimary'])
#     accuracy_distSecondaryRIGHT.append(dataCollection['accuracy_distSecondary'])
    correctiveSaccadeRIGHT.append(correctiveSaccadeValues[0])
    
    amplitudeBothEyesDictPrimaryRIGHTVariance.append(bothEyesPrimary[1])
    amplitudeBothEyesDictSecondaryRIGHTVariance.append(bothEyesSecondary[1])
    amplitudeRightEyesDictPrimaryRIGHTVariance.append(rightEyesPrimary[1])
    amplitudeRightEyesDictSecondaryRIGHTVariance.append(rightEyesSecondary[1])
    amplitudeLeftEyesDictPrimaryRIGHTVariance.append(leftEyesPrimary[1])
    amplitudeLeftEyesDictSecondaryRIGHTVariance.append(leftEyesSecondary[1])
    saccade_latencyRIGHTVariance.append(saccadeLatencyValues[1])
    correctiveSaccadeRIGHTVariance.append(correctiveSaccadeValues[1])


    print(" ")
    
#TRY THE MEAN (add to array) - to wash out the noise! - only 5 values for each of the features 


11-29-data/Right/YeRightEye.xlsx
LOOK HERE
Eyetracker timestamp [μs]                         6
Event                                          7023
Event value                                    7025
Gaze point X [MCS norm]                          15
Gaze point Y [MCS norm]                          15
Eye movement type                                 1
Presented Stimulus name                          17
AOI hit [slothStimulus-final (1) - sloth1]     3498
AOI hit [slothStimulus-final (1) - sloth2]     3498
AOI hit [slothStimulus-final (1) - sloth3]     3498
AOI hit [slothStimulus-final (1) - sloth4]     3498
AOI hit [slothStimulus-final (1) - sloth5]     3498
AOI hit [slothStimulus-final (1) - sloth6]     3498
AOI hit [slothStimulus-final (1) - sloth7]     3498
AOI hit [slothStimulus-final (1) - sloth8]     3498
AOI hit [slothStimulus-final (1) - sloth9]     3498
AOI hit [slothStimulus-final (1) - sloth10]    3498
AOI hit [slothStimulus-final (1) - sloth11]    3498
AOI hit [catStimulus-

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 7.3770785642231855,
             'AOI hit [slothStimulus-final (1) - sloth2]': 17.623439111669857,
             'AOI hit [slothStimulus-final (1) - sloth3]': 26.732766935377228,
             'AOI hit [slothStimulus-final (1) - sloth4]': 1.1041586223874205,
             'AOI hit [slothStimulus-final (1) - sloth5]': 14.718740004802573,
             'AOI hit [slothStimulus-final (1) - sloth6]': 17.98680377899466,
             'AOI hit [slothStimulus-final (1) - sloth7]': 0.16192486822205523,
             'AOI hit [slothStimulus-final (1) - sloth8]': 0.3941256933156916,
             'AOI hit [slothStimulus-final (1) - sloth9]': 18.577340230437688,
             'AOI hit [slothStimulus-final (1) - sloth10]': 17.913644434455392,
             'AOI hit [slothStimulus-final (1) - sloth11]': 13.168360495960966,
             'AOI hit [catStimulus-final (1) - cat1]': 20.871093590611636,
             'AOI hit [catStimulus-fi

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': nan,
             'AOI hit [slothStimulus-final (1) - sloth2]': 66.5531885568354,
             'AOI hit [slothStimulus-final (1) - sloth3]': 25.775288049733515,
             'AOI hit [slothStimulus-final (1) - sloth4]': 2.272842615582537,
             'AOI hit [slothStimulus-final (1) - sloth5]': 15.246462090091697,
             'AOI hit [slothStimulus-final (1) - sloth6]': 17.176497810480733,
             'AOI hit [slothStimulus-final (1) - sloth7]': 3.276083748868186,
             'AOI hit [slothStimulus-final (1) - sloth8]': 0.6865978432295927,
             'AOI hit [slothStimulus-final (1) - sloth9]': 16.196025568423238,
             'AOI hit [slothStimulus-final (1) - sloth10]': 16.265649515334776,
             'AOI hit [slothStimulus-final (1) - sloth11]': 12.806384906278542,
             'AOI hit [catStimulus-final (1) - cat1]': 17.123456850973717,
             'AOI hit [catStimulus-final (1) - cat2]': 3

 
LEFT


...

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 0.4278283543676763,
             'AOI hit [slothStimulus-final (1) - sloth2]': 0.6436510304789873,
             'AOI hit [slothStimulus-final (1) - sloth3]': 0.2669038745425643,
             'AOI hit [slothStimulus-final (1) - sloth5]': 1.1175124985919072,
             'AOI hit [slothStimulus-final (1) - sloth11]': 1.414320430648964,
             'AOI hit [catStimulus-final (1) - cat4]': 0.914288508777223,
             'AOI hit [catStimulus-final (1) - cat5]': 1.4925535035110524,
             'AOI hit [catStimulus-final (1) - cat6]': 0.21774034194402273,
             'AOI hit [catStimulus-final (1) - cat7]': 3.5817658994140698,
             'AOI hit [catStimulus-final (1) - cat8]': 1.4873979828298396,
             'AOI hit [catStimulus-final (1) - cat9]': 1.4591537622774393,
             'AOI hit [catStimulus-final (1) - cat11]': 1.1855689632642228})

 
LEFT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 1.073274266812195,
             'AOI hit [slothStimulus-final (1) - sloth2]': 0.1900363197362538,
             'AOI hit [slothStimulus-final (1) - sloth3]': 0.7614037534303948,
             'AOI hit [slothStimulus-final (1) - sloth5]': 1.5976189327964827,
             'AOI hit [slothStimulus-final (1) - sloth11]': 3.39042544602843,
             'AOI hit [catStimulus-final (1) - cat4]': 0.9696753719796571,
             'AOI hit [catStimulus-final (1) - cat5]': 1.3435183337280472,
             'AOI hit [catStimulus-final (1) - cat6]': 0.6368920266313497,
             'AOI hit [catStimulus-final (1) - cat7]': 1.3717378296649174,
             'AOI hit [catStimulus-final (1) - cat8]': 0.4009695232963416,
             'AOI hit [catStimulus-final (1) - cat9]': 1.0711990063600922,
             'AOI hit [catStimulus-final (1) - cat11]': 2.1139167888410006})

Primary Saccade Latency (Mean and STD)
0.42389859090909066 0.1641915015643896
 
Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)
17.166666666666668 10.172129679778086
 
 


In [43]:
files = glob.glob(os.path.join(folder_names[0], '*.xlsx'))
for idx, file_path in enumerate(files):
    print(file_path)
    dataCollection = defaultdict(int) 
    dataExport = pd.read_excel(file_path)
    dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)
    primarySaccadeTime, primarySaccadeIndex,secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
    bothEyesPrimary, leftEyesPrimary, rightEyesPrimary = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
    bothEyesSecondary, leftEyesSecondary, rightEyesSecondary = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
    saccadeLatencyValues = saccadeLatency(primarySaccadeTime, times_dict)
    #dataCollection['accuracy_distPrimary'] = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
    #dataCollection['accuracy_distSecondary'] = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
    correctiveSaccadeValues = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
    amplitudeBothEyesDictPrimaryNormal.append(bothEyesPrimary[0])
    amplitudeBothEyesDictSecondaryNormal.append(bothEyesSecondary[0])
    amplitudeRightEyesDictPrimaryNormal.append(rightEyesPrimary[0])
    amplitudeRightEyesDictSecondaryNormal.append(rightEyesSecondary[0])
    amplitudeLeftEyesDictPrimaryNormal.append(leftEyesPrimary[0])
    amplitudeLeftEyesDictSecondaryNormal.append(leftEyesSecondary[0])
    saccade_latencyNormal.append(saccadeLatencyValues[0])
#     accuracy_distPrimaryRIGHT.append(dataCollection['accuracy_distPrimary'])
#     accuracy_distSecondaryRIGHT.append(dataCollection['accuracy_distSecondary'])
    correctiveSaccadeNormal.append(correctiveSaccadeValues[0])
    
    amplitudeBothEyesDictPrimaryNormalVariance.append(bothEyesPrimary[1])
    amplitudeBothEyesDictSecondaryNormalVariance.append(bothEyesSecondary[1])
    amplitudeRightEyesDictPrimaryNormalVariance.append(rightEyesPrimary[1])
    amplitudeRightEyesDictSecondaryNormalVariance.append(rightEyesSecondary[1])
    amplitudeLeftEyesDictPrimaryNormalVariance.append(leftEyesPrimary[1])
    amplitudeLeftEyesDictSecondaryNormalVariance.append(leftEyesSecondary[1])
    saccade_latencyNormalVariance.append(saccadeLatencyValues[1])
    correctiveSaccadeNormalVariance.append(correctiveSaccadeValues[1])


    print(" ")

11-29-data/Normal/SanaNormal.xlsx
LOOK HERE
Eyetracker timestamp [μs]                        16
Event                                          7091
Event value                                    7092
Gaze point X [MCS norm]                          13
Gaze point Y [MCS norm]                          13
Eye movement type                                 1
Presented Stimulus name                          15
AOI hit [slothStimulus-final (1) - sloth1]     3555
AOI hit [slothStimulus-final (1) - sloth2]     3555
AOI hit [slothStimulus-final (1) - sloth3]     3555
AOI hit [slothStimulus-final (1) - sloth4]     3555
AOI hit [slothStimulus-final (1) - sloth5]     3555
AOI hit [slothStimulus-final (1) - sloth6]     3555
AOI hit [slothStimulus-final (1) - sloth7]     3555
AOI hit [slothStimulus-final (1) - sloth8]     3555
AOI hit [slothStimulus-final (1) - sloth9]     3555
AOI hit [slothStimulus-final (1) - sloth10]    3555
AOI hit [slothStimulus-final (1) - sloth11]    3555
AOI hit [catStimulus

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 5.560777085494331,
             'AOI hit [slothStimulus-final (1) - sloth2]': 1.2431788818482843,
             'AOI hit [slothStimulus-final (1) - sloth3]': 27.832057776820204,
             'AOI hit [slothStimulus-final (1) - sloth4]': 14.639654915865016,
             'AOI hit [slothStimulus-final (1) - sloth5]': 28.91665628090242,
             'AOI hit [slothStimulus-final (1) - sloth6]': 21.434821883128283,
             'AOI hit [slothStimulus-final (1) - sloth7]': 16.93574149215741,
             'AOI hit [slothStimulus-final (1) - sloth8]': 17.980306424606205,
             'AOI hit [slothStimulus-final (1) - sloth9]': 16.550537535754113,
             'AOI hit [slothStimulus-final (1) - sloth10]': 1.1547232560964675,
             'AOI hit [slothStimulus-final (1) - sloth11]': 15.361741937747208,
             'AOI hit [catStimulus-final (1) - cat1]': 8.326296967213171,
             'AOI hit [catStimulus-final 

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 4.520261296146526,
             'AOI hit [slothStimulus-final (1) - sloth2]': 1.4164159031939683,
             'AOI hit [slothStimulus-final (1) - sloth3]': 28.30502283899657,
             'AOI hit [slothStimulus-final (1) - sloth4]': 14.547428662758412,
             'AOI hit [slothStimulus-final (1) - sloth5]': 28.745862597947255,
             'AOI hit [slothStimulus-final (1) - sloth6]': 21.602294840759207,
             'AOI hit [slothStimulus-final (1) - sloth7]': 17.423074821979146,
             'AOI hit [slothStimulus-final (1) - sloth8]': 17.96896499379893,
             'AOI hit [slothStimulus-final (1) - sloth9]': 16.789998737658888,
             'AOI hit [slothStimulus-final (1) - sloth10]': 1.04904289873689,
             'AOI hit [slothStimulus-final (1) - sloth11]': 15.516277477194388,
             'AOI hit [catStimulus-final (1) - cat1]': 8.550109265246382,
             'AOI hit [catStimulus-final (1

 
LEFT


...

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 0.8309770523062545,
             'AOI hit [slothStimulus-final (1) - sloth3]': 17.84487970726513,
             'AOI hit [slothStimulus-final (1) - sloth5]': 14.945414550974009,
             'AOI hit [slothStimulus-final (1) - sloth7]': 16.780605635226053,
             'AOI hit [catStimulus-final (1) - cat1]': 3.70307604669496,
             'AOI hit [catStimulus-final (1) - cat5]': 3.279298672329665,
             'AOI hit [catStimulus-final (1) - cat6]': 2.9780952692215013,
             'AOI hit [catStimulus-final (1) - cat7]': 3.304509875842804,
             'AOI hit [catStimulus-final (1) - cat10]': 2.183886382935243})

 
LEFT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 1.1532236027796068,
             'AOI hit [slothStimulus-final (1) - sloth3]': 23.930066419787302,
             'AOI hit [slothStimulus-final (1) - sloth5]': 14.329653622007058,
             'AOI hit [slothStimulus-final (1) - sloth7]': 26.666893709558558,
             'AOI hit [catStimulus-final (1) - cat1]': 4.1746534643722075,
             'AOI hit [catStimulus-final (1) - cat5]': 3.0114251450559615,
             'AOI hit [catStimulus-final (1) - cat6]': 3.169292819932087,
             'AOI hit [catStimulus-final (1) - cat7]': 2.773381411692985,
             'AOI hit [catStimulus-final (1) - cat10]': 1.5317623767434225})

Primary Saccade Latency (Mean and STD)
0.20798499999999998 0.16103427354928354
 
Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)
11.777777777777779 6.032831982443656
 
 


In [44]:
files = glob.glob(os.path.join(folder_names[2], '*.xlsx'))
for idx, file_path in enumerate(files):
    print(file_path)
    dataCollection = defaultdict(int) 
    dataExport = pd.read_excel(file_path)
    dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)
    primarySaccadeTime, primarySaccadeIndex,secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
    bothEyesPrimary, leftEyesPrimary, rightEyesPrimary = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
    bothEyesSecondary, leftEyesSecondary, rightEyesSecondary = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
    saccadeLatencyValues = saccadeLatency(primarySaccadeTime, times_dict)
    #dataCollection['accuracy_distPrimary'] = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
    #dataCollection['accuracy_distSecondary'] = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
    correctiveSaccadeValues = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
    amplitudeBothEyesDictPrimaryLEFT.append(bothEyesPrimary[0])
    amplitudeBothEyesDictSecondaryLEFT.append(bothEyesSecondary[0])
    amplitudeRightEyesDictPrimaryLEFT.append(rightEyesPrimary[0])
    amplitudeRightEyesDictSecondaryLEFT.append(rightEyesSecondary[0])
    amplitudeLeftEyesDictPrimaryLEFT.append(leftEyesPrimary[0])
    amplitudeLeftEyesDictSecondaryLEFT.append(leftEyesSecondary[0])
    saccade_latencyLEFT.append(saccadeLatencyValues[0])
#     accuracy_distPrimaryRIGHT.append(dataCollection['accuracy_distPrimary'])
#     accuracy_distSecondaryRIGHT.append(dataCollection['accuracy_distSecondary'])
    correctiveSaccadeLEFT.append(correctiveSaccadeValues[0])
    
    amplitudeBothEyesDictPrimaryLEFTVariance.append(bothEyesPrimary[1])
    amplitudeBothEyesDictSecondaryLEFTVariance.append(bothEyesSecondary[1])
    amplitudeRightEyesDictPrimaryLEFTVariance.append(rightEyesPrimary[1])
    amplitudeRightEyesDictSecondaryLEFTVariance.append(rightEyesSecondary[1])
    amplitudeLeftEyesDictPrimaryLEFTVariance.append(leftEyesPrimary[1])
    amplitudeLeftEyesDictSecondaryLEFTVariance.append(leftEyesSecondary[1])
    saccade_latencyLEFTVariance.append(saccadeLatencyValues[1])
    correctiveSaccadeLEFTVariance.append(correctiveSaccadeValues[1])


    print(" ")

11-29-data/Left/YeLeftEye.xlsx
LOOK HERE
Eyetracker timestamp [μs]                         6
Event                                          6759
Event value                                    6761
Gaze point X [MCS norm]                          15
Gaze point Y [MCS norm]                          15
Eye movement type                                 1
Presented Stimulus name                          17
AOI hit [slothStimulus-final (1) - sloth1]     3425
AOI hit [slothStimulus-final (1) - sloth2]     3425
AOI hit [slothStimulus-final (1) - sloth3]     3425
AOI hit [slothStimulus-final (1) - sloth4]     3425
AOI hit [slothStimulus-final (1) - sloth5]     3425
AOI hit [slothStimulus-final (1) - sloth6]     3425
AOI hit [slothStimulus-final (1) - sloth7]     3425
AOI hit [slothStimulus-final (1) - sloth8]     3425
AOI hit [slothStimulus-final (1) - sloth9]     3425
AOI hit [slothStimulus-final (1) - sloth10]    3425
AOI hit [slothStimulus-final (1) - sloth11]    3425
AOI hit [catStimulus-fi

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 7.905334633942032,
             'AOI hit [slothStimulus-final (1) - sloth2]': 17.381434778105046,
             'AOI hit [slothStimulus-final (1) - sloth3]': 29.4155353553435,
             'AOI hit [slothStimulus-final (1) - sloth4]': 19.747376896335798,
             'AOI hit [slothStimulus-final (1) - sloth5]': 10.173487499918533,
             'AOI hit [slothStimulus-final (1) - sloth6]': 6.801426237643962,
             'AOI hit [slothStimulus-final (1) - sloth7]': 7.7333782761743874,
             'AOI hit [slothStimulus-final (1) - sloth8]': 7.519674444095064,
             'AOI hit [slothStimulus-final (1) - sloth9]': 13.851052659517958,
             'AOI hit [slothStimulus-final (1) - sloth10]': 14.642191956445604,
             'AOI hit [slothStimulus-final (1) - sloth11]': 15.496671173867338,
             'AOI hit [catStimulus-final (1) - cat1]': 12.924695533198388,
             'AOI hit [catStimulus-final (

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 7.905334633942032,
             'AOI hit [slothStimulus-final (1) - sloth2]': 17.381434778105046,
             'AOI hit [slothStimulus-final (1) - sloth3]': 29.4155353553435,
             'AOI hit [slothStimulus-final (1) - sloth4]': 19.747376896335798,
             'AOI hit [slothStimulus-final (1) - sloth5]': 10.173487499918533,
             'AOI hit [slothStimulus-final (1) - sloth6]': 6.801426237643962,
             'AOI hit [slothStimulus-final (1) - sloth7]': 7.7333782761743874,
             'AOI hit [slothStimulus-final (1) - sloth8]': 7.519674444095064,
             'AOI hit [slothStimulus-final (1) - sloth9]': 13.626266098572714,
             'AOI hit [slothStimulus-final (1) - sloth10]': 15.142021444747302,
             'AOI hit [slothStimulus-final (1) - sloth11]': 14.979760622292357,
             'AOI hit [catStimulus-final (1) - cat1]': 12.665120609744159,
             'AOI hit [catStimulus-final (

 
LEFT


...

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 2.3761551917654016,
             'AOI hit [slothStimulus-final (1) - sloth2]': 3.6379979712777883,
             'AOI hit [slothStimulus-final (1) - sloth3]': 1.208288278459981,
             'AOI hit [slothStimulus-final (1) - sloth5]': 6.687696370011806,
             'AOI hit [slothStimulus-final (1) - sloth6]': 1.324059721514038,
             'AOI hit [slothStimulus-final (1) - sloth9]': 15.047380023559109,
             'AOI hit [catStimulus-final (1) - cat1]': 0.9250433559289313,
             'AOI hit [catStimulus-final (1) - cat2]': 0.28134116050982844,
             'AOI hit [catStimulus-final (1) - cat3]': 0.24567107575871203,
             'AOI hit [catStimulus-final (1) - cat4]': 2.7340690390084785,
             'AOI hit [catStimulus-final (1) - cat5]': 1.0711990063600922,
             'AOI hit [catStimulus-final (1) - cat6]': 1.8099142544430016,
             'AOI hit [catStimulus-final (1) - cat7]': 1.608

 
LEFT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': nan,
             'AOI hit [slothStimulus-final (1) - sloth2]': nan,
             'AOI hit [slothStimulus-final (1) - sloth3]': nan,
             'AOI hit [slothStimulus-final (1) - sloth5]': nan,
             'AOI hit [slothStimulus-final (1) - sloth6]': nan,
             'AOI hit [slothStimulus-final (1) - sloth9]': nan,
             'AOI hit [catStimulus-final (1) - cat1]': nan,
             'AOI hit [catStimulus-final (1) - cat2]': 0.9884927582503061,
             'AOI hit [catStimulus-final (1) - cat3]': 0.7162258855794246,
             'AOI hit [catStimulus-final (1) - cat4]': 2.1568131794238763,
             'AOI hit [catStimulus-final (1) - cat5]': 1.393500395018747,
             'AOI hit [catStimulus-final (1) - cat6]': 1.8295495148791519,
             'AOI hit [catStimulus-final (1) - cat7]': 3.538520410713288,
             'AOI hit [catStimulus-final (1) - cat8]': 2.8152217672202324,
             'AO

Primary Saccade Latency (Mean and STD)
0.21555890909090902 0.16574490870115863
 
Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)
16.25 6.466258578188781
 
 


In [45]:
print(amplitudeLeftEyesDictSecondaryNormal)

[1.7523967905696705, 12.856890023765226, 4.602891456606374, 6.06020795010807, 8.971150285769909]


In [46]:
# from scipy import stats

# amplitudeBothEyesDictPrimaryLEFT = [value for dictionary in amplitudeBothEyesDictPrimaryLEFT for value in dictionary.values()]
# amplitudeBothEyesDictSecondaryLEFT = [value for dictionary in amplitudeBothEyesDictSecondaryLEFT for value in dictionary.values()]
# amplitudeRightEyesDictPrimaryLEFT = [value for dictionary in amplitudeRightEyesDictPrimaryLEFT for value in dictionary.values()]
# amplitudeRightEyesDictSecondaryLEFT = [value for dictionary in amplitudeRightEyesDictSecondaryLEFT for value in dictionary.values()]
# amplitudeLeftEyesDictPrimaryLEFT = [value for dictionary in amplitudeLeftEyesDictPrimaryLEFT for value in dictionary.values()]
# amplitudeLeftEyesDictSecondaryLEFT = [value for dictionary in amplitudeLeftEyesDictSecondaryLEFT for value in dictionary.values()]
# saccade_latencyLEFT = [value for dictionary in saccade_latencyLEFT for value in dictionary.values()]
# accuracy_distPrimaryLEFT = [value for dictionary in accuracy_distPrimaryLEFT for value in dictionary.values()]
# accuracy_distSecondaryLEFT = [value for dictionary in accuracy_distSecondaryLEFT for value in dictionary.values()]
# correctiveSaccadeLEFT = [value for dictionary in correctiveSaccadeLEFT for value in dictionary.values()]

# amplitudeBothEyesDictPrimaryRIGHT = [value for dictionary in amplitudeBothEyesDictPrimaryRIGHT for value in dictionary.values()]
# amplitudeBothEyesDictSecondaryRIGHT= [value for dictionary in amplitudeBothEyesDictSecondaryRIGHT for value in dictionary.values()]
# amplitudeRightEyesDictPrimaryRIGHT = [value for dictionary in amplitudeRightEyesDictPrimaryRIGHT for value in dictionary.values()]
# amplitudeRightEyesDictSecondaryRIGHT= [value for dictionary in amplitudeRightEyesDictSecondaryRIGHT for value in dictionary.values()]
# amplitudeLeftEyesDictPrimaryRIGHT= [value for dictionary in amplitudeLeftEyesDictPrimaryRIGHT for value in dictionary.values()]
# amplitudeLeftEyesDictSecondaryRIGHT = [value for dictionary in amplitudeLeftEyesDictSecondaryRIGHT for value in dictionary.values()]
# saccade_latencyRIGHT= [value for dictionary in saccade_latencyRIGHT for value in dictionary.values()]
# accuracy_distPrimaryRIGHT= [value for dictionary in accuracy_distPrimaryRIGHT for value in dictionary.values()]
# accuracy_distSecondaryRIGHT= [value for dictionary in accuracy_distSecondaryRIGHT for value in dictionary.values()]
# correctiveSaccadeRIGHT= [value for dictionary in correctiveSaccadeRIGHT for value in dictionary.values()]


# amplitudeBothEyesDictPrimaryNormal = [value for dictionary in amplitudeBothEyesDictPrimaryNormal for value in dictionary.values()]
# amplitudeBothEyesDictSecondaryNormal= [value for dictionary in amplitudeBothEyesDictSecondaryNormal for value in dictionary.values()]
# amplitudeRightEyesDictPrimaryNormal = [value for dictionary in amplitudeRightEyesDictPrimaryNormal for value in dictionary.values()]
# amplitudeRightEyesDictSecondaryNormal= [value for dictionary in amplitudeRightEyesDictSecondaryNormal for value in dictionary.values()]
# amplitudeLeftEyesDictPrimaryNormal= [value for dictionary in amplitudeLeftEyesDictPrimaryNormal for value in dictionary.values()]
# amplitudeLeftEyesDictSecondaryNormal = [value for dictionary in amplitudeLeftEyesDictSecondaryNormal for value in dictionary.values()]
# saccade_latencyNormal = [value for dictionary in saccade_latencyNormal for value in dictionary.values()]
# accuracy_distPrimaryNormal = [value for dictionary in accuracy_distPrimaryNormal for value in dictionary.values()]
# accuracy_distSecondaryNormal= [value for dictionary in accuracy_distSecondaryNormal for value in dictionary.values()]
# correctiveSaccadeNormal= [value for dictionary in correctiveSaccadeNormal for value in dictionary.values()]



In [47]:
print(amplitudeBothEyesDictPrimaryLEFT)

[14.707575737355086, 16.823335976816377, 19.797054656926942, 10.318901696322804, 14.618457456215681]


In [53]:
from scipy import stats

print(amplitudeBothEyesDictPrimaryLEFT)
print(amplitudeBothEyesDictPrimaryNormal)
print(amplitudeBothEyesDictPrimaryRIGHT)
print(amplitudeRightEyesDictPrimaryNormal)
print(amplitudeRightEyesDictPrimaryLEFT)
print(amplitudeRightEyesDictPrimaryRIGHT)
print(amplitudeLeftEyesDictPrimaryLEFT)
print(amplitudeLeftEyesDictPrimaryNormal)
print(amplitudeLeftEyesDictPrimaryRIGHT)

print(" ")
print(amplitudeBothEyesDictSecondaryLEFT)
print(amplitudeBothEyesDictSecondaryNormal)
print(amplitudeBothEyesDictSecondaryRIGHT)
print(amplitudeRightEyesDictSecondaryNormal)
print(amplitudeRightEyesDictSecondaryLEFT)
print(amplitudeRightEyesDictSecondaryRIGHT)
print(amplitudeLeftEyesDictSecondaryLEFT)
print(amplitudeLeftEyesDictSecondaryNormal)
print(amplitudeLeftEyesDictSecondaryRIGHT)

print(" ")

print(saccade_latencyLEFT)
print(saccade_latencyRIGHT)
print(saccade_latencyNormal)

print(" ")

print(correctiveSaccadeLEFT)
print(correctiveSaccadeRIGHT)
print(correctiveSaccadeNormal)

[14.707575737355086, 16.823335976816377, 19.797054656926942, 10.318901696322804, 14.618457456215681]
[15.867061678679768, 20.894623027763885, 13.323190263003461, 18.245912773267285, 11.240468643671239]
[14.489408448141251, 9.857462779275464, 20.983815368052547, 11.892694033395626, 19.586392810249862]
[15.898524191766922, 20.933633220908977, 13.301773191724994, 20.243663410113697, 11.146279778420379]
[14.659228710805841, 17.0617559150093, 20.158494996011278, 10.261484262437321, 14.024845646056209]
[16.985674666435123, 7.493407408649215, 20.821289046183015, 11.95774818984483, 19.72231917140735]
[11.605953101170671, 16.586835051781563, 19.435003946003896, 5.36805060890401, 22.680601165122365]
[15.778233424876543, 21.0724644108397, 13.352341961690302, 17.908873530895708, 11.90823202066231]
[14.920729506870636, 9.997625225545763, 21.186965243256854, 11.857299749411975, 20.107640101542092]
 
[5.179497801154469, 2.496804442969761, 1.406655703598842, 6.617718966347084, 2.5958450637052253]
[1.8

In [54]:
print(amplitudeBothEyesDictPrimaryLEFT,amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)

print(" ")

# amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]


# amplitudeLeftEyesDictPrimaryLEFT = [value for value in amplitudeLeftEyesDictPrimaryLEFT if not np.isnan(value)]
# amplitudeLeftEyesDictPrimaryRIGHT = [value for value in amplitudeLeftEyesDictPrimaryRIGHT if not np.isnan(value)]
# amplitudeLeftEyesDictPrimaryNormal = [value for value in amplitudeLeftEyesDictPrimaryNormal if not np.isnan(value)]

# amplitudeRightEyesDictPrimaryLEFT = [value for value in amplitudeRightEyesDictPrimaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictPrimaryRIGHT = [value for value in amplitudeRightEyesDictPrimaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictPrimaryNormal = [value for value in amplitudeRightEyesDictPrimaryNormal if not np.isnan(value)]



stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryNormal)
print(stat, p_value)

print(" ")
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeLeftEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryNormal)
print(stat, p_value)
print(" ")

# amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]

# amplitudeLeftEyesDictSecondaryLEFT = [value for value in amplitudeLeftEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeLeftEyesDictSecondaryRIGHT = [value for value in amplitudeLeftEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeLeftEyesDictSecondaryNormal = [value for value in amplitudeLeftEyesDictSecondaryNormal if not np.isnan(value)]


stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryNormal, amplitudeLeftEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryNormal, amplitudeRightEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")

stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryNormal, amplitudeBothEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")
stat, p_value = stats.mannwhitneyu(saccade_latencyLEFT, saccade_latencyRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(saccade_latencyNormal, saccade_latencyRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(saccade_latencyLEFT, saccade_latencyNormal)
print(stat, p_value)
print(" ")

print(" ")
stat, p_value = stats.mannwhitneyu(correctiveSaccadeLEFT, correctiveSaccadeRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(correctiveSaccadeNormal, correctiveSaccadeRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(correctiveSaccadeLEFT, correctiveSaccadeNormal)
print(correctiveSaccadeNormal)
print(correctiveSaccadeLEFT)
print(stat, p_value)

[14.707575737355086, 16.823335976816377, 19.797054656926942, 10.318901696322804, 14.618457456215681] [15.867061678679768, 20.894623027763885, 13.323190263003461, 18.245912773267285, 11.240468643671239] [14.489408448141251, 9.857462779275464, 20.983815368052547, 11.892694033395626, 19.586392810249862]
14.0 0.8412698412698413
13.0 1.0
11.0 0.8412698412698413
 
12.0 1.0
14.0 0.8412698412698413
10.0 0.6904761904761905
 
12.0 1.0
14.0 0.8412698412698413
12.0 1.0
 
10.0 0.6904761904761905
17.0 0.42063492063492064
5.0 0.15079365079365079
 
6.0 0.2222222222222222
16.0 0.5476190476190477
6.0 0.2222222222222222
 
12.0 1.0
18.0 0.30952380952380953
7.0 0.30952380952380953
 
14.0 0.8412698412698413
13.0 1.0
13.0 1.0
 
 
9.0 0.5476190476190477
6.0 0.2222222222222222
[13.6, 7.75, 13.35, 11.75, 11.777777777777779]
[17.0, 12.571428571428571, 8.4, 13.647058823529411, 16.25]
19.0 0.2222222222222222


In [50]:
print(amplitudeBothEyesDictPrimaryLEFTVariance,amplitudeBothEyesDictPrimaryNormalVariance, amplitudeBothEyesDictPrimaryRIGHTVariance)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFTVariance, amplitudeBothEyesDictPrimaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryNormalVariance, amplitudeBothEyesDictPrimaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFTVariance, amplitudeBothEyesDictPrimaryNormalVariance)
print(stat, p_value)

print(" ")

amplitudeRightEyesDictSecondaryLEFTVariance = [value for value in amplitudeRightEyesDictSecondaryLEFTVariance if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHTVariance = [value for value in amplitudeRightEyesDictSecondaryRIGHTVariance if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormalVariance = [value for value in amplitudeRightEyesDictSecondaryNormalVariance if not np.isnan(value)]


amplitudeLeftEyesDictPrimaryLEFTVariance = [value for value in amplitudeLeftEyesDictPrimaryLEFTVariance if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryRIGHTVariance = [value for value in amplitudeLeftEyesDictPrimaryRIGHTVariance if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryNormalVariance = [value for value in amplitudeLeftEyesDictPrimaryNormalVariance if not np.isnan(value)]

amplitudeRightEyesDictPrimaryLEFTVariance = [value for value in amplitudeRightEyesDictPrimaryLEFTVariance if not np.isnan(value)]
amplitudeRightEyesDictPrimaryRIGHTVariance = [value for value in amplitudeRightEyesDictPrimaryRIGHTVariance if not np.isnan(value)]
amplitudeRightEyesDictPrimaryNormalVariance = [value for value in amplitudeRightEyesDictPrimaryNormalVariance if not np.isnan(value)]



stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFTVariance, amplitudeRightEyesDictPrimaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryNormalVariance, amplitudeRightEyesDictPrimaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFTVariance, amplitudeRightEyesDictPrimaryNormalVariance)
print(stat, p_value)

print(" ")
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFTVariance, amplitudeLeftEyesDictPrimaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormalVariance, amplitudeLeftEyesDictPrimaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFTVariance, amplitudeLeftEyesDictPrimaryNormalVariance)
print(stat, p_value)
print(" ")

amplitudeRightEyesDictSecondaryLEFTVariance = [value for value in amplitudeRightEyesDictSecondaryLEFTVariance if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHTVariance = [value for value in amplitudeRightEyesDictSecondaryRIGHTVariance if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormalVariance = [value for value in amplitudeRightEyesDictSecondaryNormalVariance if not np.isnan(value)]

amplitudeLeftEyesDictSecondaryLEFTVariance = [value for value in amplitudeLeftEyesDictSecondaryLEFTVariance if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryRIGHTVariance = [value for value in amplitudeLeftEyesDictSecondaryRIGHTVariance if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryNormalVariance = [value for value in amplitudeLeftEyesDictSecondaryNormalVariance if not np.isnan(value)]


stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFTVariance, amplitudeLeftEyesDictSecondaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryNormalVariance, amplitudeLeftEyesDictSecondaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFTVariance, amplitudeLeftEyesDictSecondaryNormalVariance)
print(stat, p_value)
print(" ")
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryLEFTVariance, amplitudeRightEyesDictSecondaryNormalVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryNormalVariance, amplitudeRightEyesDictSecondaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryLEFTVariance, amplitudeRightEyesDictSecondaryNormalVariance)
print(stat, p_value)
print(" ")

stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryLEFTVariance, amplitudeBothEyesDictSecondaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryNormalVariance, amplitudeBothEyesDictSecondaryRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryLEFTVariance, amplitudeBothEyesDictSecondaryNormalVariance)
print(stat, p_value)
print(" ")
stat, p_value = stats.mannwhitneyu(saccade_latencyLEFTVariance, saccade_latencyRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(saccade_latencyNormalVariance, saccade_latencyRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(saccade_latencyLEFTVariance, saccade_latencyNormalVariance)
print(stat, p_value)
print(" ")

print(" ")
stat, p_value = stats.mannwhitneyu(correctiveSaccadeLEFTVariance, correctiveSaccadeRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(correctiveSaccadeNormalVariance, correctiveSaccadeRIGHTVariance)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(correctiveSaccadeLEFTVariance, correctiveSaccadeNormalVariance)
print(correctiveSaccadeNormal)
print(correctiveSaccadeLEFT)
print(stat, p_value)

[38.36128374412047, 101.96897794002132, 73.16933747407515, 44.24020929121147, 69.21449323717569] [72.8679829158575, 49.78649064477829, 28.18522926280288, 81.63404941492526, 74.24335588282638] [103.88074921704448, 89.61593118561372, 79.14802463389417, 67.992946783684, 99.62143726877333]
6.0 0.2222222222222222
4.0 0.09523809523809523
12.0 1.0
 
8.0 0.42063492063492064
6.0 0.2222222222222222
15.0 0.6904761904761905
 
9.0 0.5476190476190477
3.0 0.05555555555555555
14.0 0.8412698412698413
 
8.0 0.42063492063492064
16.0 0.5476190476190477
3.0 0.05555555555555555
 
6.0 0.2222222222222222
12.0 1.0
6.0 0.2222222222222222
 
8.0 0.42063492063492064
14.0 0.8412698412698413
6.0 0.2222222222222222
 
16.0 0.5476190476190477
20.0 0.15079365079365079
7.0 0.30952380952380953
 
 
10.0 0.6904761904761905
7.0 0.30952380952380953
[13.6, 7.75, 13.35, 11.75, 11.777777777777779]
[17.0, 12.571428571428571, 8.4, 13.647058823529411, 16.25]
15.0 0.6904761904761905


In [51]:
print(" ")
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryLEFT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryLEFT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryLEFT)
print(stat, p_value)
print(" ")
print(" ")

stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryRIGHT, amplitudeRightEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryRIGHT, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryRIGHT, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
print(" ")

stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryNormal)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)
print(" ")

 
13.0 1.0
13.0 1.0
12.0 1.0
 
 
13.0 1.0
14.0 0.8412698412698413
13.0 1.0
 
13.0 1.0
13.0 1.0
13.0 1.0
 


In [161]:
from scipy.stats import kruskal
# print(amplitudeBothEyesDictPrimaryLEFT,amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
stat, p_value = kruskal(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryRIGHT, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryNormal)
# print(stat, p_value)

print(" ")

amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]


amplitudeLeftEyesDictPrimaryLEFT = [value for value in amplitudeLeftEyesDictPrimaryLEFT if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryRIGHT = [value for value in amplitudeLeftEyesDictPrimaryRIGHT if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryNormal = [value for value in amplitudeLeftEyesDictPrimaryNormal if not np.isnan(value)]

amplitudeRightEyesDictPrimaryLEFT = [value for value in amplitudeRightEyesDictPrimaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictPrimaryRIGHT = [value for value in amplitudeRightEyesDictPrimaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictPrimaryNormal = [value for value in amplitudeRightEyesDictPrimaryNormal if not np.isnan(value)]



stat, p_value = kruskal(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryRIGHT, amplitudeRightEyesDictPrimaryNormal)
print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryNormal)
# print(stat, p_value)

print(" ")
stat, p_value = kruskal(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryRIGHT, amplitudeLeftEyesDictPrimaryNormal)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeLeftEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryNormal)
# print(stat, p_value)
print(" ")

amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]

amplitudeLeftEyesDictSecondaryLEFT = [value for value in amplitudeLeftEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryRIGHT = [value for value in amplitudeLeftEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryNormal = [value for value in amplitudeLeftEyesDictSecondaryNormal if not np.isnan(value)]


stat, p_value = kruskal(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryRIGHT, amplitudeLeftEyesDictSecondaryNormal)
print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryNormal, amplitudeLeftEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryNormal)
# print(stat, p_value)
print(" ")

stat, p_value = kruskal(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryRIGHT, amplitudeRightEyesDictSecondaryNormal)
print(stat, p_value)


stat, p_value = kruskal(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryRIGHT, amplitudeBothEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")


stat, p_value = kruskal(saccade_latencyLEFT, saccade_latencyRIGHT, saccade_latencyNormal)
print(stat, p_value)
print(" ")


stat, p_value = kruskal(accuracy_distPrimaryLEFT, accuracy_distPrimaryRIGHT, accuracy_distPrimaryNormal)
print(stat, p_value)
print(" ")

stat, p_value = kruskal(accuracy_distSecondaryLEFT, accuracy_distSecondaryRIGHT, accuracy_distSecondaryNormal)
print(stat, p_value)
print(" ")

stat, p_value = kruskal(correctiveSaccadeLEFT, correctiveSaccadeRIGHT, correctiveSaccadeNormal)
print(stat, p_value)
print(" ")



0.2447423739480273 0.8848198739283947
 
1.0815862208137899 0.5822862519033402
 
 
1.1641699508178651 0.558732208367647
 
2.110325426827825 0.34813577633671344
0.5785525543837647 0.7488052989969515
 
1.7298079351455131 0.4210919891714934
 
0.3360880037038214 0.8453166383687722
 
0.9540031342681914 0.6206415566641306
 
10.930428196332416 0.004231435007569993
 


In [162]:
from scipy.stats import wilcoxon

print(amplitudeBothEyesDictPrimaryLEFT,amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
stat, p_value = wilcoxon(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)

print(" ")

amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]


amplitudeLeftEyesDictPrimaryLEFT = [value for value in amplitudeLeftEyesDictPrimaryLEFT if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryRIGHT = [value for value in amplitudeLeftEyesDictPrimaryRIGHT if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryNormal = [value for value in amplitudeLeftEyesDictPrimaryNormal if not np.isnan(value)]

amplitudeRightEyesDictPrimaryLEFT = [value for value in amplitudeRightEyesDictPrimaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictPrimaryRIGHT = [value for value in amplitudeRightEyesDictPrimaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictPrimaryNormal = [value for value in amplitudeRightEyesDictPrimaryNormal if not np.isnan(value)]



stat, p_value = wilcoxon(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeRightEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryNormal)
print(stat, p_value)

print(" ")
stat, p_value = wilcoxon(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeLeftEyesDictPrimaryNormal, amplitudeLeftEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryNormal)
print(stat, p_value)
print(" ")

amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]

amplitudeLeftEyesDictSecondaryLEFT = [value for value in amplitudeLeftEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryRIGHT = [value for value in amplitudeLeftEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryNormal = [value for value in amplitudeLeftEyesDictSecondaryNormal if not np.isnan(value)]


stat, p_value = wilcoxon(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeLeftEyesDictSecondaryNormal, amplitudeLeftEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")
stat, p_value =wilcoxon(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeRightEyesDictSecondaryNormal, amplitudeRightEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")

stat, p_value = wilcoxon(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeBothEyesDictSecondaryNormal, amplitudeBothEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")
stat, p_value = wilcoxon(saccade_latencyLEFT, saccade_latencyRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(saccade_latencyNormal, saccade_latencyRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(saccade_latencyLEFT, saccade_latencyNormal)
print(stat, p_value)
print(" ")
stat, p_value = wilcoxon(accuracy_distPrimaryLEFT, accuracy_distPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(accuracy_distPrimaryNormal, accuracy_distPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(accuracy_distPrimaryLEFT, accuracy_distPrimaryNormal)
print(stat, p_value)
print(" ")
stat, p_value = wilcoxon(accuracy_distSecondaryLEFT, accuracy_distSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(accuracy_distSecondaryNormal, accuracy_distSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(accuracy_distSecondaryLEFT, accuracy_distSecondaryNormal)
print(stat, p_value)

print(" ")
stat, p_value = wilcoxon(correctiveSaccadeLEFT, correctiveSaccadeRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(correctiveSaccadeNormal, correctiveSaccadeRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(correctiveSaccadeLEFT, correctiveSaccadeNormal)
print(correctiveSaccadeNormal)
print(correctiveSaccadeLEFT)
print(stat, p_value)

[7.905334633942032, 17.381434778105046, 29.4155353553435, 19.747376896335798, 10.173487499918533, 6.801426237643962, 7.7333782761743874, 7.519674444095064, 13.851052659517958, 14.642191956445604, 15.496671173867338, 12.924695533198388, 5.6968654961516, 16.259921268855024, 23.513122995108052, 21.909296183266633, 15.020333630327656, 21.560429178894605, 16.65955454989199, 5.3977792711777575, 16.849943032547642, 17.107161171003323, 0.4003521919650133, 18.09838019884712, 16.640672639249747, 0.6009277768624848, 23.327565553813553, 1.4873979828298396, 18.854921382282686, 17.687999593206424, 31.80627830391118, 16.85088307657873, 0.6898319803870897, 1.155793244433404, 18.09838019884712, 15.694188089694785, 30.903176662968207, 19.113456863013553, 19.726215179267097, 32.813336688397584, 17.139463787513474, 20.347048844812, 31.416255835665197, 17.260865415413985, 1.7655573415719328, 16.83325950656756, 30.27090314731632, 32.72295241551905, 30.99049848590557, 20.953670841321237, 17.18433608162732, 1

ValueError: The samples x and y must have the same length.

In [127]:
# for key in plusLenseRight175.keys():  # Assuming keys are the same for all dictionaries
#     for i in range(len(dataCollection)):
#         for j in range(i + 1, len(dataCollection)):
#             group1 = list(dataCollection[i][key].values())
        
#             group2 = list(dataCollection[j][key].values())
#             # Perform the Kolmogorov-Smirnov test
# #             stat, p_value = stats.ks_2samp(group1, group2)
# #             print(f"KS test for Key {key} between {i} and {j}: Statistic={stat}, p-value={p_value}")
#             stat, p_value = stats.mannwhitneyu(group1, group2)


#             # Print the results
#             print(f"Mann test for Key {key} between {i} and {j}: Statistic={stat}, p-value={p_value}")

Mann test for Key amplitudeBothEyesDictPrimary between 0 and 1: Statistic=244.0, p-value=0.9719131339533064
Mann test for Key amplitudeBothEyesDictPrimary between 0 and 2: Statistic=213.0, p-value=0.6706993838713493
Mann test for Key amplitudeBothEyesDictPrimary between 1 and 2: Statistic=204.0, p-value=0.5196684087258145
Mann test for Key amplitudeRightEyeDictPrimary between 0 and 1: Statistic=248.0, p-value=0.897278961260083
Mann test for Key amplitudeRightEyeDictPrimary between 0 and 2: Statistic=235.0, p-value=0.9322314445142659
Mann test for Key amplitudeRightEyeDictPrimary between 1 and 2: Statistic=210.0, p-value=0.6184292290191917
Mann test for Key amplitudeLeftEyeDictPrimary between 0 and 1: Statistic=238.0, p-value=0.9345240995873741
Mann test for Key amplitudeLeftEyeDictPrimary between 0 and 2: Statistic=195.0, p-value=0.3883987275726807
Mann test for Key amplitudeLeftEyeDictPrimary between 1 and 2: Statistic=200.0, p-value=0.4586676508683464
Mann test for Key amplitudeBothE

The Kolmogorov-Smirnov (KS) test can be used to check the normality of your data, and if you find that your data is not normally distributed, you might choose to use the Mann-Whitney U test, which is a non-parametric test for comparing two independent samples. Here's how you can use these tests:

In [ ]:
times_dict = {'AOI hit [slothStimulus-final (1) - sloth1]' : 0,
           'AOI hit [slothStimulus-final (1) - sloth2]': 2.5,
           'AOI hit [slothStimulus-final (1) - sloth3]': 5,
           'AOI hit [slothStimulus-final (1) - sloth4]': 7.5,
           'AOI hit [slothStimulus-final (1) - sloth5]': 10,
           'AOI hit [slothStimulus-final (1) - sloth6]': 12.5,
           'AOI hit [slothStimulus-final (1) - sloth7]': 15,
           'AOI hit [slothStimulus-final (1) - sloth8]': 17.5,
           'AOI hit [slothStimulus-final (1) - sloth9]': 20,
           'AOI hit [slothStimulus-final (1) - sloth10]': 22.5,
           'AOI hit [slothStimulus-final (1) - sloth11]': 25,
           'AOI hit [catStimulus-final (1) - cat1]': 0,
           'AOI hit [catStimulus-final (1) - cat2]': 2.5,
           'AOI hit [catStimulus-final (1) - cat3]': 5,
           'AOI hit [catStimulus-final (1) - cat4]': 7.5,
           'AOI hit [catStimulus-final (1) - cat5]': 10,
           'AOI hit [catStimulus-final (1) - cat6]': 12.5,
           'AOI hit [catStimulus-final (1) - cat7]': 15,
           'AOI hit [catStimulus-final (1) - cat8]': 17.5,
           'AOI hit [catStimulus-final (1) - cat9]': 20,
           'AOI hit [catStimulus-final (1) - cat10]': 22.5,
           'AOI hit [catStimulus-final (1) - cat11]': 25}

#     file_path1 = 'plusLenses/amblypiaStudySanaLeft4.xlsx'
dataExport = pd.read_excel('plusLenses/amblypiaStudySanaRight4.xlsx')
dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)

primarySaccadeTime, primarySaccadeIndex, secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = secondarySaccadeTimeAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTime)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
saccade_latency = saccadeLatency(primarySaccadeTime, times_dict)
accuracy_dist = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
accuracy_dist = accuracySecondarySaccadeTime(secondarySaccadesTime, dfSaccade)
accuracy_dist = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
correctiveSaccade = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
correctiveSaccade = correctiveSaccadeTime(secondarySaccadesTime, primarySaccadeIndex, dfSaccade)



### Extracting relevant features from data metric

### TODO: Identify number of saccades in each AOI manually and where they are

In [ ]:
saccade_metrics_location = defaultdict(int)
saccade_metrics_number = defaultdict(int)

columns_to_keep = ['Gaze point X (MCSnorm)', 'Gaze point Y (MCSnorm)', 'Gaze point left X (MCSnorm)', 
                          'Gaze point left Y (MCSnorm)', 'Gaze point right X (MCSnorm)', 'Gaze point right Y (MCSnorm)']
for i in times_dict.keys():
    all_possible_hits = dfSaccade[dfSaccade[i] == np.max(dfSaccade[i])]

    total_saccades = 0
    df_saccades = pd.DataFrame()
    for index, row in all_possible_hits.iterrows(): 
        time = row['Eyetracker timestamp']
        #adding latency
        if (time/1000000 > (times_dict[i])+0.2) and (time/1000000 < (times_dict[i] + 2.5)):
            df_saccades = df_saccades.append(row[columns_to_keep], ignore_index=True)
        
            total_saccades +=1
    saccade_metrics_number[i] = total_saccades
    saccade_metrics_location[i] = df_saccades
  

saccade_metrics_location
saccade_metrics_number


In [ ]:
# dataMetricSaccades = dataMetric.iloc[:,list(range(520, dataMetric.shape[1]))]


In [ ]:
# dataMetricSaccades[['Number_of_saccades_in_AOI.bottomLeftCat1',
#        'Number_of_saccades_in_AOI.bottomLeftSloth',
#        'Number_of_saccades_in_AOI.bottomRightCat',
#        'Number_of_saccades_in_AOI.bottomRightCat2',
#        'Number_of_saccades_in_AOI.bottomRightSloth',
#        'Number_of_saccades_in_AOI.centerCat',
#        'Number_of_saccades_in_AOI.centerCat2',
#        'Number_of_saccades_in_AOI.centerSloth',
#        'Number_of_saccades_in_AOI.topCenterCat',
#        'Number_of_saccades_in_AOI.topCenterSloth',
#        'Number_of_saccades_in_AOI.topLeftCat',
#        'Number_of_saccades_in_AOI.topRightCat',
#        'Number_of_saccades_in_AOI.topRightSloth',
#        'Number_of_saccades_in_AOI.topRightSloth1',
#        'Time_to_entry_saccade.bottomLeftCat1']]

In [ ]:
# dataMetricSaccades['Time_to_entry_saccade.centerCat2']